# 0. Bibliotecas y carga de datos

In [1]:
import os
import numpy as np
import pandas as pd
import pyspark
from multiprocessing import Pool
from googletrans import Translator
from textdistance import levenshtein
import math as mt
pd.set_option('max_columns',500)
pd.set_option('max_rows',500)
#from pandas_ods_reader import read_ods

ModuleNotFoundError: No module named 'googletrans'

In [2]:
data_path = '../data'

In [3]:
ratings = pd.read_csv(os.path.join(data_path,'ratings.csv'))

In [4]:
ratings.shape

(981756, 3)

In [5]:
ratings.sample(5)

,book_id,user_id,rating
210702,2109,14586,3
909893,9223,40234,2
698579,7028,5549,3
137190,1373,6247,3
902934,9150,18199,3


In [6]:
#se manipulo manualmente ods to csv

In [136]:
ratings_libros_usuario = pd.read_csv(os.path.join(data_path,'ratings_libros_usuario_tarea.csv'))

In [137]:
ratings_libros_usuario.sample(5)

,title,author,avg rating,rating,date read,date added
52,La inteligencia emocional,"Goleman, Daniel *",4.02,really liked it,"Jan 20, 2019","Dec 27, 2018"
45,El llano en llamas,"Rulfo, Juan",4.19,really liked it,not set,"Mar 30, 2018"
17,"A Game of Thrones (A Song of Ice and Fire, #1)","Martin, George R.R.",4.45,it was amazing,not set,"Mar 30, 2018"
104,Tokio blues. Norwegian Wood,"Murakami, Haruki",4.03,it was ok,"Aug 19, 2018","Jul 30, 2018"
60,Alicia en el país de las maravillas. A través ...,"Carroll, Lewis",4.06,really liked it,not set,"Mar 30, 2018"


In [139]:
ratings_libros_usuario.drop_duplicates().shape, ratings_libros_usuario.shape

((106, 6), (106, 6))

In [165]:
ratings_libros_usuario[ratings_libros_usuario.title.map(lambda z: 'cchi' in z.lower())]

,title,author,avg rating,rating,date read,date added,stars_rating,book_id_search
76,Pinocchio,"Collodi, Carlo",3.85,really liked it,not set,"Mar 30, 2018",3,180617.0


In [9]:
catalogo_usuario_tarea =pd.read_csv(os.path.join(data_path,'catalogo_usuario_tarea.csv'))

In [10]:
catalogo_usuario_tarea

,estrellas,rating
0,1,did not like it
1,2,it was ok
2,3,liked it
3,4,really liked it
4,5,it was amazing


In [11]:
books = pd.read_csv(os.path.join(data_path,'books.csv'))

In [12]:
books.sample(5)

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
3721,3722,25242224,25242224,43024334,49,1609452860,9.781609e+12,"Elena Ferrante, Ann Goldstein",2014.0,Storia della bambina perduta,The Story of the Lost Child (The Neapolitan No...,eng,4.39,21358,29371,2790,185,496,2930,9876,15884,https://images.gr-assets.com/books/1499781984m...,https://images.gr-assets.com/books/1499781984s...
3233,3234,8252,8252,1366740,62,60885386,9.780061e+12,"Laura Ingalls Wilder, Garth Williams",1933.0,Farmer Boy,"Farmer Boy (Little House, #3)",NaN,4.05,39863,45289,1525,569,1980,10002,15023,17715,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
2275,2276,16134782,16134782,21962320,25,NaN,NaN,R.K. Lilley,2012.0,In Flight,"In Flight (Up in the Air, #1)",eng,4.10,64984,65654,3181,1655,3308,10492,21693,28506,https://images.gr-assets.com/books/1397321579m...,https://images.gr-assets.com/books/1397321579s...
271,272,6952,6952,1172473,177,552995878,9.780553e+12,"Laura Esquivel, Thomas Christensen, Carol Chr...",1989.0,Como agua para chocolate,Like Water for Chocolate,en-US,3.94,263199,285469,6559,8887,15941,60147,100011,100483,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
7848,7849,18422,18422,19842,36,310214696,9.780310e+12,"Ben Carson, Cecil Murphey",1990.0,Gifted Hands: The Ben Carson Story,Gifted Hands: The Ben Carson Story,eng,4.21,9871,12017,1607,167,412,1785,4035,5618,https://images.gr-assets.com/books/1331248043m...,https://images.gr-assets.com/books/1331248043s...


## 1. Utilizar el archivo ratings_libros_usuario_tarea para agregar dichos ratings al archivo de ratings csv.

#### Tratamiento de columnas

In [140]:
ratings_libros_usuario.columns.to_list()

['title ', 'author ', 'avg rating ', 'rating  ', 'date read ', 'date added ']

In [141]:
ratings_libros_usuario = ratings_libros_usuario.rename(columns = {x:x.strip() for x in ratings_libros_usuario.columns.to_list()})

In [142]:
ratings_libros_usuario.columns.to_list()

['title', 'author', 'avg rating', 'rating', 'date read', 'date added']

In [143]:
d = catalogo_usuario_tarea.to_dict()['rating']
d = {v: k for k, v in d.items()}
d

{'did not like it': 0,
 'it was ok ': 1,
 'liked it ': 2,
 'really liked it ': 3,
 'it was amazing ': 4}

#### Asignar rating correcto

In [144]:
ratings_libros_usuario['stars_rating'] = ratings_libros_usuario['rating'].map(lambda x: d[x])

In [145]:
ratings_libros_usuario.sample(5)

,title,author,avg rating,rating,date read,date added,stars_rating
34,The Shadow of the Wind (The Cemetery of Forgot...,"Ruiz Zafón, Carlos",4.27,it was amazing,not set,"Mar 30, 2018",4
7,What I Talk About When I Talk About Running,"Murakami, Haruki",3.85,it was amazing,not set,"Mar 30, 2018",4
9,"El psicoanalista (El psicoanalista, #1)","Katzenbach, John",4.13,it was amazing,"Apr 17, 2018","Mar 30, 2018",4
18,Los Románov: 1613-1918,"Montefiore, Simon Sebag *",3.95,it was amazing,"Apr 05, 2019","Feb 02, 2019",4
15,Harry Potter and the Order of the Phoenix (Har...,"Rowling, J.K.",4.50,it was amazing,not set,"Mar 30, 2018",4


In [146]:
### generar id de usuario nuevo 
54126 in set(ratings.user_id.unique())

False

#### id_book

In [147]:
#no hancen merge
aux = ratings_libros_usuario.merge(books[['book_id','title']],how='left')
aux.sample(10)

,title,author,avg rating,rating,date read,date added,stars_rating,book_id
78,El Sueño de una Noche de Verano - La Fierecill...,"Shakespeare, William",3.78,really liked it,not set,"Mar 30, 2018",3,NaN
29,La historiadora,"Kostova, Elizabeth",3.78,it was amazing,not set,"Mar 30, 2018",4,NaN
97,Voices from Chernobyl: The Oral History of a N...,"Alexievich, Svetlana",4.41,liked it,"Oct 07, 2019","Sep 16, 2019",2,NaN
14,Basada en hechos reales,"Vigan, Delphine de",3.89,it was amazing,"Jul 23, 2018","Jun 26, 2018",4,NaN
85,Eclipse,"Meyer, Stephenie",3.70,liked it,not set,"Mar 30, 2018",2,NaN
39,Love in the Time of Cholera,"Garcí­a Márquez, Gabriel",3.92,it was amazing,not set,"Mar 30, 2018",4,NaN
88,El extranjero,"Camus, Albert",3.99,liked it,not set,"Mar 30, 2018",2,NaN
26,Como agua para chocolate,"Esquivel, Laura",3.95,it was amazing,"Feb 03, 2019","Sep 07, 2018",4,NaN
45,El llano en llamas,"Rulfo, Juan",4.19,really liked it,not set,"Mar 30, 2018",3,NaN
81,Cementerio de animales,"King, Stephen *",4.00,really liked it,"May 26, 2019","Apr 17, 2019",3,NaN


In [23]:
#solo 7 hacen match
aux.book_id.isna().value_counts()

True     99
False     7
Name: book_id, dtype: int64

##### test translator y levenshtein

In [25]:
translator = Translator()

In [26]:
translation = translator.translate("Der Himmel ist blau und ich mag Bananen", dest='en')
print(translation.text)

The sky is blue and I like bananas


In [32]:
titulo = aux.title.sample(1).iloc[0]
titulo

'Fantastic Beasts - The Crimes of Grindelwald: The Original Screenplay '

In [33]:
translation = translator.translate(titulo, dest='en')

In [34]:
translation = translation.text.lower()
translation

'fantastic beasts - the crimes of grindelwald: the original screenplay'

In [49]:
books_id_title = books[['title','book_id']].copy()

In [50]:
#funcion multiproceso
def make_pool(func, params, threads):
    pool = Pool(threads)
    data = pool.imap(func, params)
    pool.close()
    pool.join()
    del pool
    return [x for x in data]

In [70]:
titles = books_id_title.title.values

In [71]:
titles = list(titles)


In [72]:
titles = [(x.lower(),translation) for x in titles]

In [73]:
titles[0]

('the hunger games (the hunger games, #1)',
 'fantastic beasts - the crimes of grindelwald: the original screenplay')

In [74]:
def similarity(tuple_titles):
    
    return levenshtein.normalized_similarity(tuple_titles[0],tuple_titles[1])

In [56]:
similarity(titles[0])

0.18840579710144922

In [57]:
list_lev = make_pool(similarity,titles,50)

In [59]:
books_id_title['levenstain_test'] = pd.Series(list_lev)

In [93]:
def search_id_book_title(titulo):
    tresh = .38
    translation = translator.translate(titulo, dest='en')
    translation = translation.text.lower()
    books_id_title['levenstain_test'] = books_id_title.title.map(lambda z: levenshtein.normalized_similarity(translation,z))
    ax = books_id_title[books_id_title['levenstain_test']>tresh].sort_values(ascending=False,by='levenstain_test').reset_index(drop=True)
    if ax.empty:
        return np.nan
    else:
        #print(ax.title.iloc[0],ax.levenstain_test.iloc[0] )
        return ax.book_id.iloc[0],ax.title.iloc[0],ax.levenstain_test.iloc[0]

    

In [83]:
def search_id_book_title_multiprocessing(titulo):
    tresh = .49
    translation = translator.translate(titulo, dest='en')
    translation = translation.text.lower()
    print(translation)
    titles = books_id_title.title.values

    titles = list(titles)

    titles = [(x.lower(),translation) for x in titles]
    list_lev = make_pool(similarity,titles,50)

    books_id_title['levenstain_test'] = pd.Series(list_lev)
    display()
    ax = books_id_title[books_id_title['levenstain_test']>tresh].sort_values(ascending=False,by='levenstain_test').reset_index(drop=True)
    if ax.empty:
        return np.nan,np.nan,np.nan,titulo
    else:
        #print(ax.title.iloc[0],ax.levenstain_test.iloc[0] )
        return ax.book_id.iloc[0],ax.title.iloc[0],ax.levenstain_test.iloc[0],titulo


In [88]:
l_resulst_translate_lev = []
for i,x in ratings_libros_usuario.iterrows():
    print(x.title)
    titulo = x.title
    r = search_id_book_title_multiprocessing(titulo)
    l_resulst_translate_lev.append(r)
    #break

El Avaro 
the miser
Estudio en escarlata 
study in scarlet
El principito 
the little prince
Cuando Hitler robó el conejo rosa 
when hitler stole the pink rabbit
The Metamorphosis 
the metamorphosis
Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)
harry potter and the prisoner of azkaban (harry potter, #3)
The Girl with the Dragon Tattoo (Millennium, #1)
the girl with the dragon tattoo (millennium, #1)
What I Talk About When I Talk About Running 
what i talk about when i talk about running
Fantastic Beasts and Where to Find Them: The Original Screenplay (Fantastic Beasts: The Original Screenplay, #1)
fantastic beasts and where to find them: the original screenplay (fantastic beasts: the original screenplay, #1)
El psicoanalista (El psicoanalista, #1)
the psychoanalyst (the psychoanalyst, # 1)
El nombre del viento (Crónica del asesino de reyes, #1)
the name of the wind (chronicle of the killer of kings, # 1)
Mira si yo te querré 
see if i will love you
Silk 
silk
El clan del o

In [152]:
aux = pd.DataFrame(l_resulst_translate_lev, columns=['book_id','result','levenshtein_similarity','input']).sort_values(ascending=False,by='levenshtein_similarity')
aux

,book_id,result,levenshtein_similarity,input
31,17245.0,Dracula,1.000000,Dracula
23,136251.0,Harry Potter and the Deathly Hallows (Harry Po...,1.000000,Harry Potter and the Deathly Hallows (Harry Po...
25,4381.0,Fahrenheit 451,1.000000,Fahrenheit 451
26,6952.0,Like Water for Chocolate,1.000000,Como agua para chocolate
27,899492.0,The Tell-Tale Heart,1.000000,The Tell-Tale Heart
28,3950967.0,The Tales of Beedle the Bard,1.000000,The Tales of Beedle the Bard
29,10692.0,The Historian,1.000000,La historiadora
30,45685.0,The Canterville Ghost,1.000000,The Canterville Ghost
33,483445.0,The Magical Worlds of Harry Potter: A Treasury...,1.000000,The Magical Worlds of Harry Potter: A Treasury...
75,10592.0,Carrie,1.000000,Carrie


In [154]:
##manipular a mano y descartar los que no tienen nada que ver
results=[['Dracula ', 17245.0],
       ['Harry Potter and the Deathly Hallows (Harry Potter, #7)',
        136251.0],
       ['Fahrenheit 451 ', 4381.0],
       ['Como agua para chocolate ', 6952.0],
       ['The Tell-Tale Heart ', 899492.0],
       ['The Tales of Beedle the Bard ', 3950967.0],
       ['La historiadora ', 10692.0],
       ['The Canterville Ghost ', 45685.0],
       ['The Magical Worlds of Harry Potter: A Treasury of Myths, Legends, and Fascinating Facts ',
        483445.0],
       ['Carrie ', 10592.0],
       ['A Thousand Splendid Suns ', 128029.0],
       ["Harry Potter and the Sorcerer's Stone (Harry Potter, #1)", 3.0],
       ['Love in the Time of Cholera ', 9712.0],
       ['La soledad de los números primos ', 6613956.0],
       ['The Princess and the Pea ', 576570.0],
       ['Usted puede sanar su vida ', 129603.0],
       ['The Martian Chronicles ', 76778.0],
       ['Twilight (Twilight, #1)', 41865.0],
       ['El evangelio según Jesucristo ', 28859.0],
       ['Perfume: The Story of a Murderer ', 343.0],
       ['The Neverending Story ', 27712.0],
       ['Pinocchio ', 180617.0],
       ['The Girl with the Dragon Tattoo (Millennium, #1)', 2429135.0],
       ['La Búsqueda ', 7076706.0],
       ['El nombre de la rosa ', 119073.0],
       ['Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)',
        5.0],
       ['Silk ', 61264.0],
       ['What I Talk About When I Talk About Running ', 2195464.0],
       ['The Metamorphosis ', 485894.0],
       ['El principito ', 157993.0],
       ['Harry Potter and the Order of the Phoenix (Harry Potter, #5)',
        2.0],
       ['A Game of Thrones (A Song of Ice and Fire, #1)', 13496.0],
       ['It ', 18342.0],#####hasta aquí todos tienen similarity  1
       ['The Shadow of the Wind (The Cemetery of Forgotten Books, #1)',
        1232.0],
       ['Pedro Páramo ', 38787.0],
       ['Estudio en escarlata ', 102868.0],
       ['Momo ', np.nan],
       ['A la sombra del ángel ', np.nan],
       ['La inteligencia emocional ', 6486483.0],
       ['La casa de los espíritus ', 9328.0],
       ['El Avaro ', np.nan],
       ['The Pillars of the Earth (Kingsbridge, #1)', 5043.0],
       ['Las Mil y Una Noches ', 93101.0],
       ['El curioso incidente del perro a medianoche ', 1618.0],
       ['Cien años de soledad ', 320.0],
       ['Caim ', 7011225.0],
       ['Viaje Al Centro de la Tierra ', np.nan],
       ['Fantastic Beasts - The Crimes of Grindelwald: The Original Screenplay ',
        30065028.0],
       ['La ciudad de las bestias ', np.nan],
       ['Las deudas del cuerpo ', np.nan],
       ['Harry Potter y el legado maldito. Partes uno y dos ',
        29056083.0],
       ['El llano en llamas ', np.nan],
       ['Alicia en el país de las maravillas. A través del espejo y lo que Alicia encontró allí ',
        83346.0],
       ['Cometas en el cielo ', np.nan],
       ['Bella y oscura ', np.nan],
       ['El resplandor ', np.nan],
       ['El diablo de los números ', np.nan],
       ['Los Ninos de Irena ', np.nan],
       ['Cementerio de animales ', 77262.0],
       ['El caballero de la armadura oxidada ', 73385.0],
       ['El nombre del viento (Crónica del asesino de reyes, #1)',
        186074.0],
       ['Sangre en el Nilo ', np.nan],
       ['The Halloween Tree ', np.nan],
       ['La noche de Tlatelolco ', np.nan],
       ['La niña perdida (La amiga estupenda, #4)', 27419654.0],
       ['Historias de mujeres ', np.nan],
       ['El cuento de la isla desconocida ', np.nan],
       ['Ciudades de papel ', 6442769.0],
       ['El clan del oso cavernario (Los hijos de la tierra, #1)',
        np.nan],
       ['Relato de un náufrago ', np.nan],
       ['Bonsai ', np.nan],
       ['Donde habitan los ángeles ', np.nan],
       ['Fantastic Beasts and Where to Find Them: The Original Screenplay (Fantastic Beasts: The Original Screenplay, #1)',
        30065028.0],
       ['Fuego y sangre (La historia de los Targaryen, #1)', np.nan],
       ['El extranjero ', np.nan],
       ['El psicoanalista (El psicoanalista, #1)', np.nan],
       ['Mi negro pasado (Como agua para chocolate, #3)', 6952.0],
       ['Un mal nombre (La amiga estupenda, #2)', np.nan],
       ["La amiga estupenda (L'amica geniale #1)", np.nan],
       ['La verdad sobre el caso Harry Quebert ', np.nan],
       ['Nada se opone a la noche ', np.nan],
       ['Tokio blues. Norwegian Wood ', 11297.0],
       ['El temor de un hombre sabio (Crónica del asesino de Reyes, #2)',
        np.nan],
       ['Maldicion de RA, La ', np.nan],
       ['Mira si yo te querré ', np.nan],
       ['Basada en hechos reales ', np.nan],
       ['Un mundo sin fin ', np.nan],
       ['Huesos de lagartija ', np.nan],
       ['El gato del Dalai Lama ', np.nan],
       ['El Sueño de una Noche de Verano - La Fierecilla Domada ',
        1622.0],
       ['Cuando Hitler robó el conejo rosa ', np.nan],
       ["Bibbi Bokken's Magic Library ", np.nan],
       ['Los Románov: 1613-1918 ', np.nan],
       ['Vita Brevis: A Letter to St Augustine ', np.nan],
       ['te amo... pero soy feliz sin ti ', np.nan],
       ['Ready Player One (Ready Player One, #1)', np.nan],
       ['La ridícula idea de no volver a verte ', np.nan],
       ['Las mujeres que aman demasiado ', np.nan],
       ['Nahui Olin, la mujer del sol ', np.nan],
       ['El horror de Dunwich ', np.nan],
       ['Eclipse ', np.nan],
       ['Muchas vidas, muchos maestros ', np.nan],
       ['Juan Salvador Gaviota ', np.nan],
       ['Jaque al Psicoanalista (El psicoanalista, #2)', np.nan],
       ['Voices from Chernobyl: The Oral History of a Nuclear Disaster ',
        np.nan],
       ['A Lupita le gustaba planchar ', np.nan]]

In [155]:
results = pd.DataFrame(results)

In [156]:
results.rename(columns={0:'title',1:'book_id_search'},inplace=True)

In [157]:
ratings_libros_usuario = ratings_libros_usuario.merge(results)

In [158]:
##quitar espacios
ratings_libros_usuario['title'] = ratings_libros_usuario.title.map(lambda z: z.strip())

In [159]:
##los que hacen match ya estan, excepto Momo
ratings_libros_usuario.merge(books[['title','book_id']])

,title,author,avg rating,rating,date read,date added,stars_rating,book_id_search,book_id
0,The Metamorphosis,"Kafka, Franz",3.83,it was amazing,not set,"Mar 30, 2018",4,485894.0,485894
1,Harry Potter and the Prisoner of Azkaban (Harr...,"Rowling, J.K.",4.57,it was amazing,not set,"Mar 30, 2018",4,5.0,5
2,"The Girl with the Dragon Tattoo (Millennium, #1)","Larsson, Stieg",4.14,it was amazing,not set,"Mar 30, 2018",4,2429135.0,2429135
3,What I Talk About When I Talk About Running,"Murakami, Haruki",3.85,it was amazing,not set,"Mar 30, 2018",4,2195464.0,2195464
4,Silk,"Baricco, Alessandro",3.85,it was amazing,not set,"Mar 30, 2018",4,61264.0,61264
5,Harry Potter and the Order of the Phoenix (Har...,"Rowling, J.K.",4.50,it was amazing,not set,"Mar 30, 2018",4,2.0,2
6,"A Game of Thrones (A Song of Ice and Fire, #1)","Martin, George R.R.",4.45,it was amazing,not set,"Mar 30, 2018",4,13496.0,13496
7,Harry Potter and the Deathly Hallows (Harry Po...,"Rowling, J.K.",4.62,it was amazing,not set,"Mar 30, 2018",4,136251.0,136251
8,Perfume: The Story of a Murderer,"Süskind, Patrick",4.02,it was amazing,"Jul 04, 2019","Aug 28, 2018",4,343.0,343
9,Fahrenheit 451,"Bradbury, Ray",3.99,it was amazing,not set,"Mar 30, 2018",4,4381.0,4381


In [162]:
ratings_libros_usuario.shape, ratings_libros_usuario.drop_duplicates().shape

((106, 8), (106, 8))

In [131]:
ratings_libros_usuario[ratings_libros_usuario.title=='Momo']

,title,author,avg rating,rating,date read,date added,stars_rating,book_id_search
35,Momo,"Ende, Michael",4.32,it was amazing,not set,"Mar 30, 2018",4.0,NaN


In [170]:
ratings_libros_usuario.at[35,'book_id_search'] = 68811

In [171]:
ratings_libros_usuario[ratings_libros_usuario.title=='Momo']

,title,author,avg rating,rating,date read,date added,stars_rating,book_id_search
35,Momo,"Ende, Michael",4.32,it was amazing,not set,"Mar 30, 2018",4,68811.0


In [177]:
##solo hay book_id menores a 10000
ratings.book_id.describe()

count    981756.000000
mean       4943.275636
std        2873.207415
min           1.000000
25%        2457.000000
50%        4921.000000
75%        7414.000000
max       10000.000000
Name: book_id, dtype: float64

In [187]:
##Asignar nuevo id a los que no se encontraron
nuevo_id = 10001
for i,r in ratings_libros_usuario.iterrows():
    #print(r.book_id_search)
    if mt.isnan(r.book_id_search):
        print(r.book_id_search)
        print(i)
        ratings_libros_usuario.at[i,'book_id_search'] = nuevo_id
        nuevo_id+=1

nan
0
nan
3
nan
9
nan
11
nan
13
nan
14
nan
16
nan
18
nan
19
nan
20
nan
21
nan
22
nan
36
nan
41
nan
44
nan
45
nan
49
nan
51
nan
53
nan
54
nan
57
nan
58
nan
62
nan
63
nan
64
nan
65
nan
67
nan
71
nan
72
nan
73
nan
74
nan
77
nan
82
nan
83
nan
84
nan
85
nan
86
nan
88
nan
89
nan
90
nan
91
nan
92
nan
93
nan
94
nan
95
nan
97
nan
99
nan
100
nan
102
nan
105


### Asignar nuevo id de usuario

In [190]:
ratings.user_id.describe()

count    981756.000000
mean      25616.759933
std       15228.338826
min           1.000000
25%       12372.000000
50%       25077.000000
75%       38572.000000
max       53424.000000
Name: user_id, dtype: float64

In [191]:
ratings_libros_usuario['user_id'] = 53425

In [198]:
ratings_usuario = ratings_libros_usuario[['user_id','stars_rating','book_id_search']].rename(columns={'book_id_search':'book_id','stars_rating':'rating'})

In [203]:
ratings_usuario

,user_id,rating,book_id
0,53425,4,10001.0
1,53425,4,102868.0
2,53425,4,157993.0
3,53425,4,10002.0
4,53425,4,485894.0
5,53425,4,5.0
6,53425,4,2429135.0
7,53425,4,2195464.0
8,53425,4,30065028.0
9,53425,4,10003.0


In [200]:
##Agregarlos a los datos

In [201]:
ratings_ = pd.concat([ratings_usuario,ratings])

In [206]:
ratings_[ratings_.user_id==53425]##revisamos los datos nuevos

,user_id,rating,book_id
0,53425,4,10001.0
1,53425,4,102868.0
2,53425,4,157993.0
3,53425,4,10002.0
4,53425,4,485894.0
5,53425,4,5.0
6,53425,4,2429135.0
7,53425,4,2195464.0
8,53425,4,30065028.0
9,53425,4,10003.0


In [212]:
ratings_.to_csv('ratings_generados_tarea4.csv')

In [2]:
import pandas as pd
import pyspark

In [3]:
ratings_ = pd.read_csv('ratings_generados_tarea4.csv')

In [70]:
books = pd.read_csv('../data/books.csv')

In [73]:
ctbooks = books[['book_id','title']]

# Generar al menos 15 recomendaciones de libros que no haya leido el usuario del archivo ratings_libros_usuario_tarea

In [4]:

pyspark.__version__

'3.0.1'

In [5]:
sc = pyspark.SparkContext(appName ="tarea")

In [6]:
sc

<SparkContext master=local[*] appName=tarea>

In [7]:
sc.version

'3.0.1'

In [8]:
sc.pythonVer

'3.8'

In [9]:
sc.defaultParallelism

8

In [10]:
num_users = ratings_.user_id.nunique()
num_books = ratings_.book_id.nunique()
num_rating = ratings_.shape[0]

sparsity = 100*(1.0 - (num_rating)/(num_users*num_books))

print(sparsity)

99.81787409816343


In [11]:
from pyspark.sql import SparkSession

In [12]:
spark = SparkSession.builder\
                    .master('local[*]') \
                    .appName('first_spark') \
                    .getOrCreate()

In [13]:
spark

In [14]:
spark.catalog.listTables()

[]

In [15]:
ratings = spark.createDataFrame(ratings_)

In [16]:
ratings.show()

+----------+-------+------+-----------+
|Unnamed: 0|user_id|rating|    book_id|
+----------+-------+------+-----------+
|         0|  53425|     4|    10001.0|
|         1|  53425|     4|   102868.0|
|         2|  53425|     4|   157993.0|
|         3|  53425|     4|    10002.0|
|         4|  53425|     4|   485894.0|
|         5|  53425|     4|        5.0|
|         6|  53425|     4|  2429135.0|
|         7|  53425|     4|  2195464.0|
|         8|  53425|     4|3.0065028E7|
|         9|  53425|     4|    10003.0|
|        10|  53425|     4|   186074.0|
|        11|  53425|     4|    10004.0|
|        12|  53425|     4|    61264.0|
|        13|  53425|     4|    10005.0|
|        14|  53425|     4|    10006.0|
|        15|  53425|     4|        2.0|
|        16|  53425|     4|    10007.0|
|        17|  53425|     4|    13496.0|
|        18|  53425|     4|    10008.0|
|        19|  53425|     4|    10009.0|
+----------+-------+------+-----------+
only showing top 20 rows



In [17]:
ratings.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- rating: long (nullable = true)
 |-- book_id: double (nullable = true)



In [18]:
ratings = ratings.select(ratings.user_id.cast("integer"), ratings.book_id.cast("integer"), ratings.rating.cast("double"))


In [19]:
ratings.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- rating: double (nullable = true)



In [20]:
ratings.createOrReplaceTempView("ratings")

In [21]:
spark.catalog.listTables()

[Table(name='ratings', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [75]:
ctbooks = spark.createDataFrame(ctbooks)

ctbooks.show()

+--------+--------------------+
| book_id|               title|
+--------+--------------------+
| 2767052|The Hunger Games ...|
|       3|Harry Potter and ...|
|   41865|Twilight (Twiligh...|
|    2657|To Kill a Mocking...|
|    4671|    The Great Gatsby|
|11870085|The Fault in Our ...|
|    5907|          The Hobbit|
|    5107|The Catcher in th...|
|     960|Angels & Demons  ...|
|    1885| Pride and Prejudice|
|   77203|     The Kite Runner|
|13335037|Divergent (Diverg...|
|    5470|                1984|
|    7613|         Animal Farm|
|   48855|The Diary of a Yo...|
| 2429135|The Girl with the...|
| 6148028|Catching Fire (Th...|
|       5|Harry Potter and ...|
|      34|The Fellowship of...|
| 7260188|Mockingjay (The H...|
+--------+--------------------+
only showing top 20 rows



In [76]:
ctbooks.printSchema()

root
 |-- book_id: long (nullable = true)
 |-- title: string (nullable = true)



In [77]:
ctbooks.createOrReplaceTempView("ctbooks")

In [ ]:
ctbooks = spark.createDataFrame(ctbooks_)

ctbooks.show()

ctbooks.printSchema()

ctbooks = ctbooks.select(ctbooks.user_id.cast("integer"), ctbooks.book_id.cast("integer"), ctbooks.rating.cast("double"))


ctbooks.printSchema()

ctbooks.createOrReplaceTempView("ctbooks")

In [22]:
from pyspark.sql.functions import min ,max, avg, sum

In [23]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [24]:
(train, test) = ratings.randomSplit([0.75, 0.25], seed = 24)

In [25]:
help(ALS)

Help on class ALS in module pyspark.ml.recommendation:

class ALS(pyspark.ml.wrapper.JavaEstimator, _ALSParams, pyspark.ml.util.JavaMLWritable, pyspark.ml.util.JavaMLReadable)
 |  ALS(rank=10, maxIter=10, regParam=0.1, numUserBlocks=10, numItemBlocks=10, implicitPrefs=False, alpha=1.0, userCol='user', itemCol='item', seed=None, ratingCol='rating', nonnegative=False, checkpointInterval=10, intermediateStorageLevel='MEMORY_AND_DISK', finalStorageLevel='MEMORY_AND_DISK', coldStartStrategy='nan', blockSize=4096)
 |  
 |  Alternating Least Squares (ALS) matrix factorization.
 |  
 |  ALS attempts to estimate the ratings matrix `R` as the product of
 |  two lower-rank matrices, `X` and `Y`, i.e. `X * Yt = R`. Typically
 |  these approximations are called 'factor' matrices. The general
 |  approach is iterative. During each iteration, one of the factor
 |  matrices is held constant, while the other is solved for using least
 |  squares. The newly-solved factor matrix is then held constant whi

In [40]:
als = ALS(userCol="user_id", itemCol="book_id", ratingCol="rating")

In [41]:
modelo = als.fit(train)

In [42]:

predicciones = modelo.transform(test)

In [43]:
predicciones.show()

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|  35982|    148|   3.0| 3.0806258|
|  20467|    148|   3.0| 2.9251928|
|   9731|    148|   3.0| 2.9033399|
|   3087|    148|   3.0|  3.388159|
|  26629|    148|   4.0| 3.7998915|
|   3922|    148|   3.0| 3.5575104|
|  24326|    148|   5.0| 3.3275745|
|  13034|    148|   4.0| 3.5006433|
|  32055|    148|   3.0| 3.1159427|
|  20967|    148|   3.0|   4.04404|
|  10610|    148|   4.0| 3.7438827|
|  14603|    148|   4.0| 3.5203726|
|  14248|    148|   4.0|  3.529253|
|  18361|    148|   4.0| 3.6695652|
|  23576|    148|   3.0|  2.799698|
|  45554|    148|   4.0| 3.4960632|
|   8579|    148|   3.0| 2.8327124|
|  22534|    148|   4.0|  3.427063|
|  11272|    148|   3.0| 2.9317694|
|  11692|    148|   4.0| 3.6549544|
+-------+-------+------+----------+
only showing top 20 rows



In [44]:
predicciones.select(avg("rating")).show()

+------------------+
|       avg(rating)|
+------------------+
|3.8557228719418837|
+------------------+



In [45]:
predicciones.select(avg("prediction")).show()

+---------------+
|avg(prediction)|
+---------------+
|            NaN|
+---------------+



In [46]:
predicciones.createOrReplaceTempView("predicciones")

In [33]:
spark.catalog.listTables()

[Table(name='predicciones_1', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='ratings', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [47]:
query = " select * from predicciones where prediction = 'NaN' "
spark.sql(query).show()

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|  14972|   4818|   3.0|       NaN|
|  50577|   4935|   3.0|       NaN|
|  30101|   4935|   4.0|       NaN|
|  48753|   5518|   3.0|       NaN|
|  53233|   6620|   5.0|       NaN|
|  41426|   7982|   5.0|       NaN|
|  38235|   9465|   3.0|       NaN|
|  29201|   9465|   4.0|       NaN|
|  48173|   8932|   3.0|       NaN|
|   3961|   9182|   5.0|       NaN|
|  35351|   9564|   2.0|       NaN|
|  19146|   2572|   4.0|       NaN|
|  43690|   6482|   3.0|       NaN|
|  29721|   6482|   3.0|       NaN|
|  46494|   6482|   3.0|       NaN|
|  12600|   6559|   3.0|       NaN|
|  42598|   6622|   2.0|       NaN|
|  46523|   7879|   3.0|       NaN|
|  40502|   8222|   3.0|       NaN|
|   4046|   9517|   4.0|       NaN|
+-------+-------+------+----------+
only showing top 20 rows



In [48]:
query = " select * from predicciones  where prediction <> 'NaN' order by prediction desc "
spark.sql(query).show()

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|  15744|   6089|   5.0| 6.5546436|
|  23917|   9806|   5.0|  6.049839|
|   2821|   4154|   5.0| 5.9806542|
|  25434|   9539|   5.0|  5.943401|
|  48129|   5919|   5.0|  5.921979|
|  17290|   3628|   5.0|  5.863548|
|  48944|   4441|   5.0|  5.861889|
|  52469|    267|   5.0|  5.836901|
|  34992|   8170|   4.0| 5.8008623|
|  19474|   6876|   5.0|  5.782319|
|  34533|   2672|   5.0| 5.7750916|
|  40356|   2865|   5.0|  5.772716|
|  20082|   9522|   5.0|  5.763012|
|  33356|   9076|   5.0| 5.7510962|
|   7868|   8847|   5.0| 5.7467995|
|  50138|   5035|   5.0| 5.7326274|
|  18171|   9076|   5.0| 5.7306204|
|  51589|   6572|   5.0|  5.709619|
|  19802|   8415|   5.0|  5.705305|
|  30470|   6710|   5.0| 5.6929493|
+-------+-------+------+----------+
only showing top 20 rows



In [36]:
train.createOrReplaceTempView("train")

In [49]:
#REvisamos nuestro id nuevo
query = " select * from predicciones where user_id = 53425"
spark.sql(query).show()

+-------+-------+------+----------+
|user_id|book_id|rating|prediction|
+-------+-------+------+----------+
|  53425|  10032|   3.0|       NaN|
|  53425|  10018|   3.0|       NaN|
|  53425| 576570|   3.0|       NaN|
|  53425|  10043|   2.0|       NaN|
|  53425|  76778|   3.0|       NaN|
|  53425| 128029|   4.0|       NaN|
|  53425|  10006|   4.0|       NaN|
|  53425|  10037|   2.0|       NaN|
|  53425|  10049|   1.0|       NaN|
|  53425| 129603|   3.0|       NaN|
|  53425|  10035|   2.0|       NaN|
|  53425|  10014|   4.0|       NaN|
|  53425|  10026|   3.0|       NaN|
|  53425| 180617|   3.0|       NaN|
|  53425|2195464|   4.0|       NaN|
|  53425|  10027|   3.0|       NaN|
|  53425|    320|   4.0|  3.102512|
|  53425|  10023|   3.0|       NaN|
|  53425|  10041|   2.0|       NaN|
|  53425|  10050|   1.0|       NaN|
+-------+-------+------+----------+
only showing top 20 rows



In [39]:
## generar recomendaciones al usuario nuevo

In [116]:

userRecs = modelo.recommendForAllUsers(300)

In [117]:
#userRecs.show()

In [118]:
userRecs.createOrReplaceTempView("userRecs")

In [119]:
#query = "select user_id, explode(recommendations) as tupla from userrecs"
#spark.sql(query).show()

In [120]:
query ="SELECT user_id, booksIds_and_ratings.book_id AS book_id, booksIds_and_ratings.rating AS prediction FROM userRecs LATERAL VIEW explode(recommendations) exploded_table AS booksIds_and_ratings"
tabla_predicciones = spark.sql(query)
#tabla_predicciones.show()

In [121]:
##ponemos catalogo de libros
tabla_predicciones_ct = tabla_predicciones.join(ctbooks, tabla_predicciones.book_id==ctbooks.book_id, 'left').select(tabla_predicciones.user_id, tabla_predicciones.book_id, tabla_predicciones.prediction, ctbooks.title)
#tabla_predicciones_ct.show()

In [122]:
##peliculas que no se han visto
doby = [tabla_predicciones_ct.user_id == ratings.user_id , tabla_predicciones_ct.book_id == ratings.book_id]
union = tabla_predicciones_ct.join(ratings, doby, 'left').select(tabla_predicciones_ct.user_id, tabla_predicciones_ct.book_id, tabla_predicciones_ct.prediction, tabla_predicciones_ct.title, ratings.rating)
#union.show()

In [123]:
union.createOrReplaceTempView("union")

In [125]:
##recomendaciones con título
query = "select * from union where rating is null and user_id = 53425 and title is not null"
recomendations = spark.sql(query).toPandas()
recomendations.shape

(18, 5)

## Recomendar 15 peliculas que no ha visto al usuario del archivo

In [127]:
recomendations.head(15)

,user_id,book_id,prediction,title,rating
0,53425,9569,3.902090,Villa Incognito,NaN
1,53425,5552,3.853160,QED: The Strange Theory of Light and Matter,NaN
2,53425,5113,3.827053,Franny and Zooey,NaN
3,53425,98,3.806890,What to Expect the First Year (What to Expect),NaN
4,53425,998,3.827808,The Millionaire Next Door: The Surprising Secr...,NaN
5,53425,36,3.786592,The Lord of the Rings: Weapons and Warfare,NaN
6,53425,4122,3.987738,Built to Last: Successful Habits of Visionary ...,NaN
7,53425,250,3.868464,Tropic of Capricorn,NaN
8,53425,8698,3.948833,"So Long, and Thanks for All the Fish (Hitchhik...",NaN
9,53425,2095,3.799583,The Universe in a Nutshell,NaN


## Comparar con el wishlist

In [139]:
wl = pd.read_csv('../data/wishlist.csv')
pd.set_option('display.max_rows', 500)

In [144]:
wl

,title,author
0,Isabel Moctezuma,"Aguirre, Eugenio"
1,La música del silencio,"Rothfuss, Patrick *"
2,21 Lessons for the 21st Century,"Harari, Yuval Noah"
3,Homo Deus: A History of Tomorrow,"Harari, Yuval Noah"
4,La bailarina de Auschwitz,"Eger, Edith *"
5,Talentos ocultos,"Shetterly, Margot Lee *"
6,Interview with the Vampire (The Vampire Chroni...,"Rice, Anne"
7,Dracul,"Stoker, Dacre"
8,La ley de Herodes,"Ibargüengoitia, Jorge"
9,La maldición de Hill House,"Jackson, Shirley"


In [137]:
l_recomendaciones = recomendations.title.to_list()

In [143]:
wl[wl['title '].isin(l_recomendaciones)]

,title,author


### Hallazgos y resultados

 Le recomendamos al usuario libros de géneros como Fantasía, Magia o Sagas, es bastante bueno el modelo de recomendacion, entre la wishlist y las recomendaciones no hubo match. No pudimos observar aquellas recomendaciones a libros cuyo título no se encontró, seguramente habrían estado mucho mas parecidos a la wishlist